In [10]:
# importing the modules
import os
import cv2
import tkinter as tk
import tkinter.filedialog as fd
import tkinter.messagebox as mb
from PIL import Image
from skimage import transform, img_as_int, exposure
from skimage.feature import blob_log, blob_doh, blob_dog
import skimage.io as io
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage, spatial
from scipy.optimize import curve_fit
from cmath import inf
import sys
import Polywarp

def find_max (x,y, arr):
    
    print(np.shape(arr))
    print((arr))
    max_elem = -1
    for i in range(x-10, x+11):
        for j in range(y-10, y+11):
            if arr[j][i] > max_elem:
                max_elem =  arr[j][i]

    list_index_max =[(i,j) for i in range(x-20, x+21)  for j in range(y-20, y+21) if arr[j][i]  == max_elem]
    line, column = list_index_max[0]
    return(line, column)
# https://www.geeksforgeeks.org/displaying-the-coordinates-of-the-points-clicked-on-the-image-using-python-opencv/  
# function to display the coordinates of  of the points clicked on the image


def click_event(event, x, y, flags, params):
    global counter
    global green_coord
    global red_coord
    if event == cv2.EVENT_LBUTTONDOWN:   # checking for left mouse clicks
        
        
        print(x,y)   
        x_max,y_max = find_max(x,y,image_array)
        print(x_max, y_max)
        if counter % 2 != 0: 
            red_coord.append(x_max) 
            red_coord.append(y_max-256)
            counter = counter + 1
            print (counter)
        else:
            green_coord.append(x_max) 
            green_coord.append(y_max)
            counter = counter + 1
        cv2.circle(better_contrast_img, (x_max, y_max), 4, (0, 0, 127), 2)
    if event==cv2.EVENT_RBUTTONDOWN:
        counter = 0
        green_coord.clear()
        red_coord.clear()
        

        
        
def blob_detection(img, min_sigma, max_sigma, threshold, method=0):
    """This function is mostly used for detecting the beads in any image.

    Args:
        img_path (string): The absolute path of the input image.
        min_sigma (int): The minimum sigma, lower it is, smaller the blob will be detected.
        max_sigma (int): The maximum sigma, higher it is, bigger the blob will be detected.
        threshold (float): Higher it is, higher the intensities of blobs.
        method (int, optional): 0 for Difference of Gaussian (DoG) and 1 for Determinant of Hessian (DoH). 
        They should be applied with different combination of parameters. DoG is more suitable for fret movies,
        while DoH is more suitable for sequencing images. Defaults to 0.

    Returns:
        centers: A numpy array containing the coordinates of all the centers.
    """
    #img = io.imread(img_path)
    if method == 0:
        blob = blob_dog(
            img, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold
        )
    else:
        blob = blob_doh(
            img, min_sigma=min_sigma, max_sigma=max_sigma, threshold=threshold
        )
    i = 0
    # r = 3
    centers = []
    h, w = img.shape
    for blob in blob:
        y, x, r = blob
        if y > r and y < (h - r) and x > r and x < (w - r):
            centers.append(
                ndimage.measurements.center_of_mass(
                    img[int(y - r) : int(y + r + 1), int(x - r) : int(x + r + 1)]
                )
            )
            centers[i] = list(np.add(centers[i], [x - r, y - r]))
            i += 1
    
    return np.array(centers)
 



def show_blob_detection_res(img, min_sigma, max_sigma, threshold, method=0):
    """
    Showing the result of 'blob detection' function. Used as the same way of 'blob_detection'
    """
    fig, ax = plt.subplots()
    #img = io.imread(img_path)
    ax.imshow(img)
    if method == 0:
        res = blob_dog(
            img,
            min_sigma=min_sigma,
            max_sigma=max_sigma,
            threshold=threshold,
        )
    else:
        res = blob_doh(
            img,
            min_sigma=min_sigma,
            max_sigma=max_sigma,
            threshold=threshold,
        )
    i = 0
    CM = []
    r = 3
    [h, w] = img.shape
    for blob in res:
        y, x, r = blob
        # print(r)
        if y > r and y < (h - r) and x > r and x < (w - r):
            CM.append(
                ndimage.measurements.center_of_mass(
                    img[int(y - r) : int(y + r), int(x - r) : int(x + r)]
                )
            )
            CM[i] = list(np.add(CM[i], [y - r, x - r]))
            x1, y1 = CM[i]
            c = plt.Circle([y1, x1], 3, color="red", linewidth=1, fill=False)
            ax.add_patch(c)
            i += 1
    ax.set_axis_off()
    plt.show()

def count_nearest_pts(src, dst, radius):
    """Counting the number of nearest neighbors for each given point.

    Args:
        src (numpy array): (N, 2) shape array. Build the kd tree based on this.
        dst (numpy array): (N, 2) shape array. For each point in this array, find the nearest neighbors in src array.
        radius (int): The maximum searching radius.

    Returns:
        res, idx: res is the distance for the point and its neighbor, 'inf' means no neighbor in given search radius. 
        idx is the index for the neighbor in src array.
    """
    tree = spatial.KDTree(src)
    res, idx = tree.query(dst, k=1, distance_upper_bound=radius)
    for i in range(0, len(idx)):
        if len(np.argwhere(idx == idx[i])) > 1:
            res[i] = inf
    return res, idx

In [ ]:
root = tk.Tk()
root.attributes("-topmost", True)
root.withdraw()

path = fd.askdirectory(title = "Choose the directory containing beads movies", initialdir = "C:/Users/Anton/Documents/Jupyter home/ExampleData/Test_folder/220717_FC_Nano_200nm_Multicolor/original")

usable_green_beads = np.empty((0, 2))
usable_red_beads = np.empty((0, 2))
#print(usable_green_beads)
#print(usable_red_beads)
os.chdir(path)
green_coord = []
red_coord = []
counter = 0 
for current_dir,dirs, files in os.walk(path) :
    for el in files: 
     #print(dirs)
        if el.split('.')[-1].lower() == 'tif':

            img = io.imread(current_dir + '/' + el) #reading the stack of images
            img1 = np.mean(img, axis = 0) #averaging by the stack
            img1 = img1.astype("ushort") #turn from float format to ushort

            image_array = np.asarray(img1)
            #print(image_array)
            v_min, v_max = np.percentile(image_array, (0.2, 99.8))
            better_contrast_image_array = exposure.rescale_intensity(image_array, in_range=(v_min, v_max))
            better_contrast_img = Image.fromarray(better_contrast_image_array)
            better_contrast_img.save(current_dir + '/'+'better_contrast'+ el[:-4]+'.png')
            better_contrast_img = cv2.imread(current_dir + '/'+'better_contrast' + el[:-4]+'.png')
            #reading by cv2 to make possible circles in color

            # displaying the image

            # setting mouse handler for the image
            # and calling the click_event() function

            if counter == 0:
                cv2.namedWindow('image') 
                cv2.setMouseCallback('image', click_event)
                while(1):

                    cv2.imshow('image',better_contrast_img)

                    #if cv2.waitKey(20) & 0xFF == 27: #press Esc to quit
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break


            # wait for a key to be pressed to exit
            img1_green_channel_arr = img1 [0:256, 0:512]
            img1_red_channel_arr = img1 [256:512, 0:512]
            
            img1_red_channel = Image.fromarray(img1_red_channel_arr)
            img1_red_channel.save(current_dir + '/'+'red_channel'+ el[:-4]+'.png')

            
            img1_green_channel = Image.fromarray(img1_green_channel_arr)
            img1_green_channel.save(current_dir + '/'+'green_channel'+ el[:-4]+'.png')

            green_coord = np.reshape(green_coord,(int((counter+1)/2), 2))
            red_coord = np.reshape(red_coord,(int((counter+1)/2), 2))
            print ('green_coord', green_coord)
            print ('red_coord', red_coord)
            tform = transform.estimate_transform('polynomial', src=green_coord, dst=red_coord, order=1)
            tform_inverse = transform.estimate_transform('polynomial', src=red_coord, dst=green_coord, order=1)

            cv2.destroyAllWindows()
            red_channel_centers = blob_detection(
                     img1_red_channel_arr,
                     min_sigma=1,
                     max_sigma=10,
                     threshold=0.001,
                     )
            show_blob_detection_res(
                     img1_red_channel_arr,
                     min_sigma=1,
                     max_sigma=10,
                     threshold=0.001,
                     )

            green_channel_centers = blob_detection(
                     img1_green_channel_arr,
                     min_sigma=1,
                     max_sigma=10,
                     threshold=0.004,
                     )
            show_blob_detection_res(
                     img1_green_channel_arr,
                     min_sigma=1,
                     max_sigma=10,
                     threshold=0.004,
                     )
            green_channel_centers = tform (green_channel_centers)
            res, idx = count_nearest_pts(green_channel_centers, red_channel_centers, 2)

            ind_red = np.where(res != inf)[0]
            ind_green = idx[np.where(res != inf)]

            usable_green_beads = np.append(usable_green_beads,  green_channel_centers[ind_green], axis=0 )
            usable_red_beads = np.append(usable_red_beads, red_channel_centers[ind_red], axis=0 )

usable_green_beads = tform_inverse(usable_green_beads) #go back to the initial coordinates

order = 3
n_beads = usable_green_beads.shape[0]

kx,ky = Polywarp.polywarp(usable_red_beads[:,0],usable_red_beads[:,1],usable_green_beads[:,0],usable_green_beads[:,1],degree=order)
print(kx)
print(ky)
tr = transform.PolynomialTransform()


order1 = 2*order
pidx = 0
par = np.zeros([2,int((order1+1)*(order1+2)/2)])
for j in range(order1 + 1):
    for i in range(j + 1):
        if (j-i)<=order and i<=order:
            par[0, pidx] = kx[j - i,i]
            par[1, pidx] = ky[j - i,i]
        else:
            par[0, pidx] = 0
            par[1, pidx] = 0
        pidx += 1
tr.params = par        

kx,ky = Polywarp.polywarp(usable_green_beads[:,0],usable_green_beads[:,1],usable_red_beads[:,0],usable_red_beads[:,1],degree=order)
tr_inv = transform.PolynomialTransform()


order1 = 2*order
pidx = 0
par = np.zeros([2,int((order1+1)*(order1+2)/2)])
for j in range(order1 + 1):
    for i in range(j + 1):
        if (j-i)<=order and i<=order:
            par[0, pidx] = kx[j - i,i]
            par[1, pidx] = ky[j - i,i]
        else:
            par[0, pidx] = 0
            par[1, pidx] = 0
        pidx += 1
tr_inv.params = par    


res = np.subtract(tr(usable_green_beads),usable_red_beads)
res = np.multiply(res,res)
print('Mean squared displacement after G to R transformation:',np.sum(res)/n_beads)
res = np.subtract(usable_green_beads,tr_inv(usable_red_beads))
res = np.multiply(res,res)
print('Mean squared displacement after inverse (R to G) transformation:',np.sum(res)/n_beads)
res = np.subtract(usable_green_beads,usable_red_beads)
res = np.multiply(res,res)
print('Mean squared displacement before transformation:',np.sum(res)/n_beads)

np.save('tform_result',tr.params )
np.save('tform_result_inv',tr_inv.params)